In [30]:
!pip3 install chess --upgrade
import chess
import numpy as np


def set_piece_on_free_square(board, free_squares, piece):
    is_valid = False
    remaining_squares = free_squares.copy()
    c = 0

    while not is_valid and c<64 and len(remaining_squares)>0:
        c+=1
        square = np.random.choice(remaining_squares)
        remaining_squares.remove(square)
        board.set_piece_at(square, piece) 

        if piece.piece_type == chess.KING:
            if board.status() == chess.STATUS_OPPOSITE_CHECK:    # opposite check (kings touch)
                is_valid = False
            else:
                is_valid = True
        else:
            is_valid = board.is_valid()

        if not is_valid:
            board.remove_piece_at(square) 
    
    if c >= 64 or len(remaining_squares) <= 0:
        return None
    else:
        free_squares.remove(square)
        return square


def set_rand_piece(board, free_squares, all_pieces):
    is_valid = False
    remaining_pieces = all_pieces.copy()

    while not is_valid and len(remaining_pieces) > 0:
        piece = np.random.choice(remaining_pieces)
        remaining_squares = free_squares.copy()
        square = set_piece_on_free_square(board, remaining_squares, piece)

        if square == None: # set_piece has failed, retry
            remaining_pieces.remove(piece)
        else:
            if board.outcome() == None: # not only valid but also legal position (you have to be able to make moves, not instant end)
                free_squares.remove(square)
                return True
            else: # so remove the piece you just placed and try with another piece
                board.remove_piece_at(square)
                remaining_pieces.remove(piece)
                
    return False


def gen_rand_board_n_pieces(n_pieces, color=None):
    assert n_pieces > 2, "you cannot have less than two pieces"
    
    flag = False
    while flag == False:
        flag = True
        board = chess.Board()
        board.clear() # WHITE to move
        free_squares = list(range(64))
        set_piece_on_free_square(board, free_squares, chess.Piece(chess.KING, chess.WHITE))
        set_piece_on_free_square(board, free_squares, chess.Piece(chess.KING, chess.BLACK))

        pieces = [chess.PAWN, chess.BISHOP, chess.KNIGHT, chess.ROOK, chess.QUEEN]
        colors = [chess.WHITE, chess.BLACK]

        all_pieces = [chess.Piece(piece, color) for piece in pieces for color in colors]

        if color == None:
            np.random.choice([chess.WHITE, chess.BLACK])
            board.push(chess.Move.null())
            board.clear_stack()
        else:
            if color == chess.BLACK:
                board.push(chess.Move.null())
                board.clear_stack()

        for n in range(n_pieces-2):
            if flag == False:
                break
            else:
                flag = set_rand_piece(board, free_squares, all_pieces)
        
        if flag == False:
            print("Restarting, cannot reach n_pieces")

    return board

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
# 8-3=5 --> 5*2*500000 --> 5'000'000
total_games = 5000000
n_pieces_min = 3
n_pieces_max = 7

    for i in range(int(total_games/(2*(n_pieces_max-n_pieces_min)))):
        for color in [chess.WHITE, chess.BLACK]:
            for n_pieces in range(n_pieces_min, n_pieces_max+1):
                board = gen_rand_board_n_pieces(n_pieces, color)
                f.write(board.fen())

board

FileNotFoundError: ignored

In [15]:
# import chess.gaviota

# tablebase = chess.gaviota.PythonTablebase()
# tablebase.add_directory("tablebase/Gaviota-Tablebases/")

In [16]:
# tablebase.probe_wdl(board)

In [38]:
with open("dataset.txt", "w") as f:
    pass

def generate_dataset(total_games, n_pieces_min=3, n_pieces_max=7):
    with open("dataset.txt", "a") as f:
        for i in range(int(total_games/(2*(n_pieces_max+1-n_pieces_min)))):
            for color in [chess.WHITE, chess.BLACK]:
                for n_pieces in range(n_pieces_min, n_pieces_max+1):
                    board = gen_rand_board_n_pieces(n_pieces, color)
                    f.write(board.fen()+"\n")
                    # print(board.fen())


In [39]:
# 8-3=5 --> 5*2*00000 --> 500'000
total_games = 2*5*50000

# %timeit generate_dataset(total_games)
generate_dataset(total_games)

8/5k2/8/r7/8/5K2/8/8 w - - 0 1
k7/8/2P5/1K6/7P/8/8/8 w - - 0 1
8/3q4/8/2P5/8/4PK1k/8/8 w - - 0 1
4K3/8/3b4/6N1/5P2/4q1k1/8/8 w - - 0 1
8/2R2q2/5k2/2Q4P/1p5K/8/8/8 w - - 0 1
8/8/8/2R5/6k1/3K4/8/8 b - - 1 1
8/2K4P/4k3/8/8/8/3b4/8 b - - 1 1
4K3/5R2/8/3k4/8/8/8/5R1q b - - 1 1
8/8/3kq3/8/8/2r5/4R3/1R1K4 b - - 1 1
8/8/K2k2R1/4N2N/8/2p5/8/1Q6 b - - 1 1
